In [1]:
import numpy as np

import CAS.dmrghandler.src.dmrghandler as dmrghandler
from CAS.dmrghandler.src.dmrghandler import dmrg_calc_prepare
from pathlib import Path
import scipy
import pandas as pd
import os
import itertools
import xlsxwriter

In [2]:
def get_obt_diag_dominance(obt):
    """
    Calculate the diagonal dominance of obt
    Args:
        obt: OBT in chemist notation

    Returns: diagonal dominance ratio in terms of L2 norm

    """
    n = obt.shape[0]
    diag = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                diag[i, i] = obt[i, j]
    
    l2_diag = scipy.linalg.norm(diag, ord=None)
    l2_total = scipy.linalg.norm(obt, ord=None)
    
    return l2_diag / l2_total


def get_tbt_diag_dominance(tbt):
    """
    Calculate the diagonal dominance of tbt
    Args:
        tbt:  TBT in chemist notation

    Returns: diagonal dominance ratio in terms of L2 norm

    """
    n = tbt.shape[0]
    diag = np.zeros((n, n, n, n))
    for p in range(n):
        for q in range(n):
            for r in range(n):
                for s in range(n):
                    if p == q and r == s:
                        diag[p, q, r, s] = tbt[p, q, r, s]
                    elif p == r and q == s:
                        diag[p, q, r, s] = tbt[p, q, r, s]
                    elif p == s and q == r:
                        diag[p, q, r, s] = tbt[p, q, r, s]
    
    l2_diag = scipy.linalg.norm(diag, ord=None)
    l2_total = scipy.linalg.norm(tbt, ord=None)
    return l2_diag / l2_total

In [3]:
fcidump_path = Path("fcidumps_catalysts")
writer = pd.ExcelWriter("diag_dominance_20240619.xlsx", engine='xlsxwriter')
column_names = ["FCIDUMP file", "OBT diag dominance", "TBT diag dominance", "overall dominance" ]

data = []
fcidumps_names = []
obt_diag_dominance_list = []
tbt_diag_dominance_list = []
overall_dominance_list = []

In [4]:
for filename in os.listdir(fcidump_path):
    print(f"Current file: {filename}")
    (
    one_body_tensor,
    two_body_tensor,
    nuc_rep_energy,
    num_orbitals,
    num_spin_orbitals,
    num_electrons,
    two_S,
    two_Sz,
    orb_sym,
    extra_attributes,
    ) = dmrg_calc_prepare.load_tensors_from_fcidump(data_file_path=fcidump_path/Path(filename), molpro_orbsym_convention=True)
    
    Hobt = one_body_tensor
    Htbt = two_body_tensor
    Hobt -= 0.5 * np.einsum("prrq->pq", Htbt.copy())
    Htbt *= 0.5
    
    obt_diag_dominance = get_obt_diag_dominance(Hobt)
    tbt_diag_dominance = get_tbt_diag_dominance(Htbt)
    
    fcidumps_names.append(filename)
    obt_diag_dominance_list.append(obt_diag_dominance)
    tbt_diag_dominance_list.append(tbt_diag_dominance)
    overall_dominance_list.append(obt_diag_dominance + tbt_diag_dominance)
    
data.append(fcidumps_names)
data.append(obt_diag_dominance_list)
data.append(tbt_diag_dominance_list)
data.append(overall_dominance_list)

Current file: fcidump.24_ts_1over2_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}
Parsing fcidumps_catalysts/fcidump.24_ts_1over2_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}
Current file: fcidump.31_4a_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}
Parsing fcidumps_catalysts/fcidump.31_4a_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}
Current file: fcidump.33_2ru_III_3pl_{'default'_'6-31_G(dp)''Ru'_'lanl2tz'}
Parsing fcidumps_catalysts/fcidump.33_2ru_III_3pl_{'default'_'6-31_G(dp)''Ru'_'lanl2tz'}
Current file: fcidump.28_2_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}
Parsing fcidumps_catalysts/fcidump.28_2_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}
Current file: fcidump.49_2_{'default'_'cc-pVTZ''Ru'_'LANL2TZ'}
Parsing fcidumps_catalysts/fcidump.49_2_{'default'_'cc-pVTZ''Ru'_'LANL2TZ'}
Current file: fcidump.48_2_{'default'_'cc-pVTZ''Ru'_'LANL

In [5]:
df_1 = pd.DataFrame()
#Iteratively add data to new columns
for column_name, column_data in zip(column_names, data):
    print(column_name)
    print(column_data)
    df_1[column_name] = column_data

df_1.to_excel(writer, sheet_name="Diagonal dominance result", index=False)

FCIDUMP file
["fcidump.24_ts_1over2_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}", "fcidump.31_4a_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}", "fcidump.33_2ru_III_3pl_{'default'_'6-31_G(dp)''Ru'_'lanl2tz'}", "fcidump.28_2_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}", "fcidump.49_2_{'default'_'cc-pVTZ''Ru'_'LANL2TZ'}", "fcidump.48_2_{'default'_'cc-pVTZ''Ru'_'LANL2TZ'}", "fcidump.58_5_{'default'_'6-31__G(dp)'}", "fcidump.54_ts56_{'default'_'6-31__G(dp)'}", "fcidump.40_1_ts_{'default'_'cc-pVTZ''Ru'_'LANL2TZ'}", "fcidump.20_rc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}", 'fcidump.8_melact_6-311__G__', "fcidump.65_5_15_af_{'default'_'cc-pVTZ''Mn'_'lanl2tz'}", "fcidump.57_5_{'default'_'6-31__G(dp)'}", "fcidump.52_6acme_{'default'_'6-31__G(dp)'}", "fcidump.42_1_star_{'default'_'cc-pVTZ''Ru'_'LANL2TZ'}", "fcidump.60_5_16_{'default'_'cc-pVTZ''Mn'_'lanl2tz'}", "fcidump.1_ru_macho_{'Ru'_'cc-pV

In [6]:
writer.close()

## Sparsity counter

In [19]:
def get_obt_sparsity(obt):
    n = obt.shape[0]
    counter = 0
    for i in range(n):
        for j in range(n):
            if obt[i, j] < 1e-3:
                counter += 1
    
    return counter / n ** 2

def get_tbt_sparsity(tbt):
    n = tbt.shape[0]
    counter = 0
    for p in range(n):
        for q in range(n):
            for r in range(n):
                for s in range(n):
                    if tbt[p, q, r, s] < 1e-3:
                        counter += 1
    
    return counter / n ** 4

In [20]:
fcidump_path = Path("fcidumps_catalysts")
writer = pd.ExcelWriter("Sparsity_20240610.xlsx", engine='xlsxwriter')
column_names = ["FCIDUMP file", "OBT sparsity", "TBT sparsity", "overall sparsity" ]

data = []
fcidumps_names = []
obt_sparsity_list = []
tbt_sparsity_list = []
overall_sparsity_list = []

In [21]:
for filename in os.listdir(fcidump_path):
    print(f"Current file: {filename}")
    (
    one_body_tensor,
    two_body_tensor,
    nuc_rep_energy,
    num_orbitals,
    num_spin_orbitals,
    num_electrons,
    two_S,
    two_Sz,
    orb_sym,
    extra_attributes,
    ) = dmrg_calc_prepare.load_tensors_from_fcidump(data_file_path=fcidump_path/Path(filename), molpro_orbsym_convention=True)
    
    Hobt = one_body_tensor
    Htbt = two_body_tensor
    Hobt -= 0.5 * np.einsum("prrq->pq", Htbt.copy())
    Htbt *= 0.5
    
    obt_sparsity= get_obt_sparsity(Hobt)
    tbt_sparsity = get_tbt_sparsity(Htbt)
    
    fcidumps_names.append(filename)
    obt_sparsity_list.append(obt_sparsity)
    tbt_sparsity_list.append(tbt_sparsity)
    overall_sparsity_list.append(obt_sparsity + tbt_sparsity)
    
data.append(fcidumps_names)
data.append(obt_sparsity_list)
data.append(tbt_sparsity_list)
data.append(overall_sparsity_list)

Current file: fcidump.24_ts_1over2_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}
Parsing fcidumps_catalysts/fcidump.24_ts_1over2_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}
Current file: fcidump.31_4a_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}
Parsing fcidumps_catalysts/fcidump.31_4a_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}
Current file: fcidump.33_2ru_III_3pl_{'default'_'6-31_G(dp)''Ru'_'lanl2tz'}
Parsing fcidumps_catalysts/fcidump.33_2ru_III_3pl_{'default'_'6-31_G(dp)''Ru'_'lanl2tz'}
Current file: fcidump.28_2_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}
Parsing fcidumps_catalysts/fcidump.28_2_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}
Current file: fcidump.49_2_{'default'_'cc-pVTZ''Ru'_'LANL2TZ'}
Parsing fcidumps_catalysts/fcidump.49_2_{'default'_'cc-pVTZ''Ru'_'LANL2TZ'}
Current file: fcidump.48_2_{'default'_'cc-pVTZ''Ru'_'LANL

In [22]:
df_1 = pd.DataFrame()
#Iteratively add data to new columns
for column_name, column_data in zip(column_names, data):
    print(column_name)
    print(column_data)
    df_1[column_name] = column_data

df_1.to_excel(writer, sheet_name="Sparsity result", index=False)

writer.close()

FCIDUMP file
["fcidump.24_ts_1over2_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}", "fcidump.31_4a_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}", "fcidump.33_2ru_III_3pl_{'default'_'6-31_G(dp)''Ru'_'lanl2tz'}", "fcidump.28_2_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}", "fcidump.49_2_{'default'_'cc-pVTZ''Ru'_'LANL2TZ'}", "fcidump.48_2_{'default'_'cc-pVTZ''Ru'_'LANL2TZ'}", "fcidump.58_5_{'default'_'6-31__G(dp)'}", "fcidump.54_ts56_{'default'_'6-31__G(dp)'}", "fcidump.40_1_ts_{'default'_'cc-pVTZ''Ru'_'LANL2TZ'}", "fcidump.20_rc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}", 'fcidump.8_melact_6-311__G__', "fcidump.65_5_15_af_{'default'_'cc-pVTZ''Mn'_'lanl2tz'}", "fcidump.57_5_{'default'_'6-31__G(dp)'}", "fcidump.52_6acme_{'default'_'6-31__G(dp)'}", "fcidump.42_1_star_{'default'_'cc-pVTZ''Ru'_'LANL2TZ'}", "fcidump.60_5_16_{'default'_'cc-pVTZ''Mn'_'lanl2tz'}", "fcidump.1_ru_macho_{'Ru'_'cc-pV